<a href="https://colab.research.google.com/github/BrunoReis136/ciencias_de_dados/blob/main/ETL.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 🧰 Instalar dependências
!pip install diffusers transformers accelerate scipy safetensors opencv-python moviepy

# 🧱 Importações
import os
import cv2
from moviepy.editor import *
from PIL import Image
from diffusers import StableDiffusionControlNetPipeline, ControlNetModel, UniPCMultistepScheduler
from transformers import AutoTokenizer
import torch

# 📁 Criar diretórios
os.makedirs("frames_original", exist_ok=True)
os.makedirs("frames_transformados", exist_ok=True)

# 📽️ Upload do vídeo
from google.colab import files
uploaded = files.upload()
video_path = list(uploaded.keys())[0]

# 🎞️ Extrair frames
vidcap = cv2.VideoCapture(video_path)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(f"frames_original/frame{count:03d}.png", image)
    success, image = vidcap.read()
    count += 1
print(f"{count} frames extraídos.")

# 📦 Baixar modelo ControlNet
from diffusers.utils import load_image
controlnet = ControlNetModel.from_pretrained(
    "lllyasviel/sd-controlnet-canny", torch_dtype=torch.float16
)

pipe = StableDiffusionControlNetPipeline.from_pretrained(
    "runwayml/stable-diffusion-v1-5",
    controlnet=controlnet,
    safety_checker=None,
    torch_dtype=torch.float16
).to("cuda")
pipe.scheduler = UniPCMultistepScheduler.from_config(pipe.scheduler.config)

# 🎨 Função para transformar cada frame
from PIL import ImageFilter

prompt = input("\n Descreva as mudanças no vídeo: \n")
negative_prompt = "blurry, low quality, deformed, text, watermark"

for i in range(count):
    img = Image.open(f"frames_original/frame{i:03d}.png")
    img = img.resize((512, 512))  # redimensiona para SD
    img = img.filter(ImageFilter.EDGE_ENHANCE)  # ou canny se quiser
    output = pipe(prompt, image=img, num_inference_steps=25, negative_prompt=negative_prompt).images[0]
    output.save(f"frames_transformados/frame{i:03d}.png")

# 🎞️ Juntar os frames transformados em vídeo
image_files = [f"frames_transformados/frame{i:03d}.png" for i in range(count)]
clip = ImageSequenceClip(image_files, fps=vidcap.get(cv2.CAP_PROP_FPS))
clip.write_videofile("video_sinistro.mp4", codec="libx264")


In [ ]:
# 🧰 Instalar dependências
!pip install opencv-python moviepy

# 🧱 Importações
import os
import cv2
import numpy as np
from PIL import Image
from moviepy.editor import ImageSequenceClip
from google.colab import files

# 📁 Criar diretórios
os.makedirs("frames_original", exist_ok=True)
os.makedirs("frames_transformados", exist_ok=True)

# 📤 Upload do vídeo e personagem
print("Faça upload do VÍDEO e da IMAGEM PNG do personagem com fundo transparente.")
uploaded = files.upload()
video_path = [f for f in uploaded.keys() if f.endswith(".mp4")][0]
personagem_path = [f for f in uploaded.keys() if f.endswith(".png")][0]

# 🎞️ Extrair frames do vídeo
vidcap = cv2.VideoCapture(video_path)
fps = vidcap.get(cv2.CAP_PROP_FPS)
success, image = vidcap.read()
count = 0
while success:
    cv2.imwrite(f"frames_original/frame{count:03d}.png", image)
    success, image = vidcap.read()
    count += 1
print(f"{count} frames extraídos.")

# 🧟‍♂️ Carregar personagem com fundo transparente
personagem = Image.open(personagem_path).convert("RGBA")
personagem = personagem.resize((150, 300))  # ajuste conforme desejar

# 🌫️ Função para aplicar neblina no frame (OpenCV)
def aplicar_neblina_cv2(frame_np):
    h, w, _ = frame_np.shape
    neblina = np.random.normal(loc=200, scale=30, size=(h, w, 3)).astype(np.uint8)
    neblina = cv2.GaussianBlur(neblina, (101, 101), 0)
    return cv2.addWeighted(frame_np, 0.85, neblina, 0.15, 0)

# 🖼️ Processar cada frame
for i in range(count):
    # Carregar frame original
    frame_path = f"frames_original/frame{i:03d}.png"
    frame = Image.open(frame_path).convert("RGBA")

    # Adicionar personagem (simula movimento vindo de longe)
    x = 200  # posição horizontal fixa
    y = 400 - i * 4  # personagem se aproxima
    frame_editado = frame.copy()
    frame_editado.paste(personagem, (x, y), personagem)

    # Converter para NumPy para aplicar neblina com OpenCV
    frame_np = np.array(frame_editado.convert("RGB"))
    frame_np = aplicar_neblina_cv2(frame_np)

    # Salvar frame final
    final = Image.fromarray(frame_np)
    final.save(f"frames_transformados/frame{i:03d}.png")

print("✅ Todos os frames foram processados.")

# 📽️ Recriar vídeo final
image_files = [f"frames_transformados/frame{i:03d}.png" for i in range(count)]
clip = ImageSequenceClip(image_files, fps=fps)
clip.write_videofile("video_sinistro_final.mp4", codec="libx264")

print("✅ Vídeo gerado com sucesso: video_sinistro_final.mp4")
